#Challenge 1

##Method 2

###Train and Test functions

In [1]:
import torch.nn as nn
def train(model, device, train_loader, optimizer, epoch, display=True):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = nn.functional.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
    if display:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()))
 
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.functional.cross_entropy(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return 100. * correct / len(test_loader.dataset)

#MODEL 1

###Model 1 with Relu Activation( 3 convolutional layers and 2 fully connected layers)

In [ ]:
#mycode
import torch
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU(inplace=True)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(in_features=128 * 4 * 4, out_features=512)
        self.dropout1 = nn.Dropout(p=0.5)
        self.relu4 = nn.ReLU(inplace=True)

        self.fc2 = nn.Linear(in_features=512, out_features=10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)

        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.bn2(x)

        x = self.relu2(x)

        x = self.pool2(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.dropout1(x)

        x = self.relu3(x)
        x = self.pool3(x)

        x = x.view(x.size(0), -1)

        x = self.fc1(x)
        x = self.dropout1(x)
        x = self.relu4(x)

        x = self.fc2(x)

        return x



####Training and Testing on Model 1 with no enhancements

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset

  
from torchvision import datasets, transforms
normalize = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))

transform_val = transforms.Compose([transforms.ToTensor(), normalize]) #careful to keep this one same
transform_train = transforms.Compose([transforms.ToTensor(), normalize]) 

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

##### Cifar Data
cifar_data = datasets.CIFAR10(root='.',train=True, transform=transform_train, download=True)
    
#We need two copies of this due to weird dataset api 
cifar_data_val = datasets.CIFAR10(root='.',train=True, transform=transform_val, download=True)
    

accs = []

for seed in range(25):
  prng = RandomState(seed)
  random_permute = prng.permutation(np.arange(0, 1000))
  classes =  prng.permutation(np.arange(0,10))
  indx_train = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
  indx_val = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])


  train_data = Subset(cifar_data, indx_train)
  val_data = Subset(cifar_data_val, indx_val)

  print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))
  
  train_loader = torch.utils.data.DataLoader(train_data,
                                             batch_size=128, 
                                             shuffle=True)

  val_loader = torch.utils.data.DataLoader(val_data,
                                           batch_size=128, 
                                           shuffle=False)
  

  model = Net()
  model.to(device)
  optimizer = torch.optim.SGD(model.parameters(),lr=0.01, momentum=0.9,
                              weight_decay=0.0005)
  for epoch in range(100):
    train(model, device, train_loader, optimizer, epoch, display=epoch%5==0)
    
  accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over 25 instances: %.2f +- %.2f'%(accs.mean(),accs.std()))


Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.447087
Train Epoch: 5 [0/50 (0%)]	Loss: 0.424754
Train Epoch: 10 [0/50 (0%)]	Loss: 0.132379
Train Epoch: 15 [0/50 (0%)]	Loss: 0.375092
Train Epoch: 20 [0/50 (0%)]	Loss: 0.018141
Train Epoch: 25 [0/50 (0%)]	Loss: 0.024136
Train Epoch: 30 [0/50 (0%)]	Loss: 0.001367
Train Epoch: 35 [0/50 (0%)]	Loss: 0.018814
Train Epoch: 40 [0/50 (0%)]	Loss: 0.000005
Train Epoch: 45 [0/50 (0%)]	Loss: 0.000204
Train Epoch: 50 [0/50 (0%)]	Loss: 0.000040
Train Epoch: 55 [0/50 (0%)]	Loss: 0.000272
Train Epoch: 60 [0/50 (0%)]	Loss: 0.000106
Train Epoch: 65 [0/50 (0%)]	Loss: 0.000011
Train Epoch: 70 [0/50 (0%)]	Loss: 0.000004
Train Epoch: 75 [0/50 (0%)]	Loss: 0.000856
Train Epoch: 80 [0/50 (0%)]	Loss: 0.007205
Train Epoch: 85 [0/50 (0%)]	Loss: 0.001573
Train Epoch: 90 [0/50 (0%)]	Loss: 0.000007
Train Epoch: 95 [0/50 (0%)]	Loss: 0.000021

Test set: Ave

###Ensemble learning, Data augumentation, learning rate schduelling with SGD optimizer

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms

# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 25
accs = []

for seed in range(num_models):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = Net()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0005)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70, 90], gamma=0.1)

    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()

    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))


Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.168715
Train Epoch: 5 [0/50 (0%)]	Loss: 1.112639
Train Epoch: 10 [0/50 (0%)]	Loss: 0.778934
Train Epoch: 15 [0/50 (0%)]	Loss: 0.661175
Train Epoch: 20 [0/50 (0%)]	Loss: 0.495780
Train Epoch: 25 [0/50 (0%)]	Loss: 0.602570
Train Epoch: 30 [0/50 (0%)]	Loss: 0.584825
Train Epoch: 35 [0/50 (0%)]	Loss: 0.502925
Train Epoch: 40 [0/50 (0%)]	Loss: 0.443110
Train Epoch: 45 [0/50 (0%)]	Loss: 0.443299
Train Epoch: 50 [0/50 (0%)]	Loss: 0.527118
Train Epoch: 55 [0/50 (0%)]	Loss: 0.453128
Train Epoch: 60 [0/50 (0%)]	Loss: 0.514670
Train Epoch: 65 [0/50 (0%)]	Loss: 0.421344
Train Epoch: 70 [0/50 (0%)]	Loss: 0.385790
Train Epoch: 75 [0/50 (0%)]	Loss: 0.392821
Train Epoch: 80 [0/50 (0%)]	Loss: 0.354773
Train Epoch: 85 [0/50 (0%)]	Loss: 0.396532
Train Epoch: 90 [0/50 (0%)]	Loss: 0.467678
Train Epoch: 95 [0/50 (0%)]	Loss: 0.449588

Test set: Ave

####Traning and testing on Model 1 with ADADELTA optimizer and Relu activation

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms

# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 25
accs = []

for seed in range(25):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = Net()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adadelta(model.parameters(), lr=0.1, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70, 90], gamma=0.1)

    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()

    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))


Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.200924
Train Epoch: 5 [0/50 (0%)]	Loss: 1.300126
Train Epoch: 10 [0/50 (0%)]	Loss: 0.459093
Train Epoch: 15 [0/50 (0%)]	Loss: 0.350475
Train Epoch: 20 [0/50 (0%)]	Loss: 0.355721
Train Epoch: 25 [0/50 (0%)]	Loss: 0.300286
Train Epoch: 30 [0/50 (0%)]	Loss: 0.235461
Train Epoch: 35 [0/50 (0%)]	Loss: 0.179940
Train Epoch: 40 [0/50 (0%)]	Loss: 0.149089
Train Epoch: 45 [0/50 (0%)]	Loss: 0.147237
Train Epoch: 50 [0/50 (0%)]	Loss: 0.097671
Train Epoch: 55 [0/50 (0%)]	Loss: 0.152691
Train Epoch: 60 [0/50 (0%)]	Loss: 0.070802
Train Epoch: 65 [0/50 (0%)]	Loss: 0.067393
Train Epoch: 70 [0/50 (0%)]	Loss: 0.078352
Train Epoch: 75 [0/50 (0%)]	Loss: 0.103242
Train Epoch: 80 [0/50 (0%)]	Loss: 0.102826
Train Epoch: 85 [0/50 (0%)]	Loss: 0.070886
Train Epoch: 90 [0/50 (0%)]	Loss: 0.121598
Train Epoch: 95 [0/50 (0%)]	Loss: 0.110942

Test set: Ave


####Training and Testing on Model 1 with Adam optimizer with Relu activation

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms

# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 25
accs = []

for seed in range(num_models):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = Net()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=0.0005)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70, 90], gamma=0.1)

    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()

    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))


Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.838223
Train Epoch: 5 [0/50 (0%)]	Loss: 82.919746
Train Epoch: 10 [0/50 (0%)]	Loss: 2.548420
Train Epoch: 15 [0/50 (0%)]	Loss: 0.827579
Train Epoch: 20 [0/50 (0%)]	Loss: 0.832308
Train Epoch: 25 [0/50 (0%)]	Loss: 0.822140
Train Epoch: 30 [0/50 (0%)]	Loss: 0.704765
Train Epoch: 35 [0/50 (0%)]	Loss: 0.569628
Train Epoch: 40 [0/50 (0%)]	Loss: 0.499830
Train Epoch: 45 [0/50 (0%)]	Loss: 0.584597
Train Epoch: 50 [0/50 (0%)]	Loss: 0.588275
Train Epoch: 55 [0/50 (0%)]	Loss: 0.556254
Train Epoch: 60 [0/50 (0%)]	Loss: 0.582798
Train Epoch: 65 [0/50 (0%)]	Loss: 0.614775
Train Epoch: 70 [0/50 (0%)]	Loss: 0.587008
Train Epoch: 75 [0/50 (0%)]	Loss: 0.513019
Train Epoch: 80 [0/50 (0%)]	Loss: 0.608023
Train Epoch: 85 [0/50 (0%)]	Loss: 0.621916
Train Epoch: 90 [0/50 (0%)]	Loss: 0.472925
Train Epoch: 95 [0/50 (0%)]	Loss: 0.548048

Test set: Av

### Model 1 with Mish activation(3 Convolutional layers and 2 fully connected layers)

In [ ]:
#mycode
import torch
import torch.nn as nn

class NetMish(nn.Module):
    def __init__(self):
        super(NetMish, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.mish = nn.Mish()

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU(inplace=True)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(in_features=128 * 4 * 4, out_features=512)
        self.dropout1 = nn.Dropout(p=0.5)
        self.relu4 = nn.ReLU(inplace=True)

        self.fc2 = nn.Linear(in_features=512, out_features=10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)

        x = self.mish(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.bn2(x)

        x = self.mish(x)

        x = self.pool2(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.dropout1(x)

        x = self.mish(x)
        x = self.pool3(x)

        x = x.view(x.size(0), -1)

        x = self.fc1(x)
        x = self.dropout1(x)
        x = self.mish(x)

        x = self.fc2(x)

        return x



####Training and Testing on Model 1 with Mish Activation and ADADELTA optimizer

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms

# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 25
accs = []

for seed in range(25):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = NetMish()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adadelta(model.parameters(), lr=0.1, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70, 90], gamma=0.1)

    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()

    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))


100%|██████████| 170498071/170498071 [00:05<00:00, 29725198.54it/s]


Extracting ./cifar-10-python.tar.gz to .
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.697130
Train Epoch: 5 [0/50 (0%)]	Loss: 0.782274
Train Epoch: 10 [0/50 (0%)]	Loss: 0.536322
Train Epoch: 15 [0/50 (0%)]	Loss: 0.436159
Train Epoch: 20 [0/50 (0%)]	Loss: 0.266625
Train Epoch: 25 [0/50 (0%)]	Loss: 0.197887
Train Epoch: 30 [0/50 (0%)]	Loss: 0.204576
Train Epoch: 35 [0/50 (0%)]	Loss: 0.110024
Train Epoch: 40 [0/50 (0%)]	Loss: 0.133132
Train Epoch: 45 [0/50 (0%)]	Loss: 0.077788
Train Epoch: 50 [0/50 (0%)]	Loss: 0.084725
Train Epoch: 55 [0/50 (0%)]	Loss: 0.094156
Train Epoch: 60 [0/50 (0%)]	Loss: 0.098055
Train Epoch: 65 [0/50 (0%)]	Loss: 0.063898
Train Epoch: 70 [0/50 (0%)]	Loss: 0.047475
Train Epoch: 75 [0/50 (0%)]	Loss: 0.060604
Train Epoch: 80 [0/50 (0%)]	Loss: 0.089316
Train Epoch: 85 [0/50 (0%)]	Loss: 0.128589
Train Epoch: 90 [0/50 (0%)]	Loss: 0.097007
Train Epoch: 95 [0/50 (0%)]	Loss: 0.059110


/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.3881, Accuracy: 338/400 (84.50%)

Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.468055
Train Epoch: 5 [0/50 (0%)]	Loss: 1.385040
Train Epoch: 10 [0/50 (0%)]	Loss: 0.711154
Train Epoch: 15 [0/50 (0%)]	Loss: 0.634292
Train Epoch: 20 [0/50 (0%)]	Loss: 0.609418
Train Epoch: 25 [0/50 (0%)]	Loss: 0.560226
Train Epoch: 30 [0/50 (0%)]	Loss: 0.445048
Train Epoch: 35 [0/50 (0%)]	Loss: 0.471970
Train Epoch: 40 [0/50 (0%)]	Loss: 0.501672
Train Epoch: 45 [0/50 (0%)]	Loss: 0.360072
Train Epoch: 50 [0/50 (0%)]	Loss: 0.374834
Train Epoch: 55 [0/50 (0%)]	Loss: 0.363171
Train Epoch: 60 [0/50 (0%)]	Loss: 0.343903
Train Epoch: 65 [0/50 (0%)]	Loss: 0.308375
Train Epoch: 70 [0/50 (0%)]	Loss: 0.275852
Train Epoch: 75 [0/50 (0%)]	Loss: 0.320445
Train Epoch: 80 [0/50 (0%)]	Loss: 0.360867
Train Epoch: 85 [0/50 (0%)]	Loss: 0.355609
Train Epoch: 90 [0/50 (0%)]	Loss: 0.317742
Train Epoch: 95 [0/50 (0%)]	Loss: 0.371626

Test set: Average loss: 0.59

####Training and Testing on Model 1 with Mish Activation and ADAM optimizer

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms

# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 25
accs = []

for seed in range(num_models):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = NetMish()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=0.0005)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70, 90], gamma=0.1)

    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()

    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))


Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.286644
Train Epoch: 5 [0/50 (0%)]	Loss: 201.339722
Train Epoch: 10 [0/50 (0%)]	Loss: 19.077681
Train Epoch: 15 [0/50 (0%)]	Loss: 33.351246
Train Epoch: 20 [0/50 (0%)]	Loss: 13.822009
Train Epoch: 25 [0/50 (0%)]	Loss: 8.450606
Train Epoch: 30 [0/50 (0%)]	Loss: 6.354549
Train Epoch: 35 [0/50 (0%)]	Loss: 2.863804
Train Epoch: 40 [0/50 (0%)]	Loss: 1.963470
Train Epoch: 45 [0/50 (0%)]	Loss: 0.964662
Train Epoch: 50 [0/50 (0%)]	Loss: 0.821055
Train Epoch: 55 [0/50 (0%)]	Loss: 0.762093
Train Epoch: 60 [0/50 (0%)]	Loss: 0.796928
Train Epoch: 65 [0/50 (0%)]	Loss: 0.690448
Train Epoch: 70 [0/50 (0%)]	Loss: 1.318594
Train Epoch: 75 [0/50 (0%)]	Loss: 0.678243
Train Epoch: 80 [0/50 (0%)]	Loss: 0.849905
Train Epoch: 85 [0/50 (0%)]	Loss: 0.728962
Train Epoch: 90 [0/50 (0%)]	Loss: 1.180013
Train Epoch: 95 [0/50 (0%)]	Loss: 0.526361

Test set

###Model 1 with Silu activation( 3 Convolutional layers and 2 fully connected layers)

In [ ]:
#mycode
import torch
import torch.nn as nn

class NetSilu(nn.Module):
    def __init__(self):
        super(NetSilu, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.silu = nn.SiLU()

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(in_features=128 * 4 * 4, out_features=512)
        self.dropout1 = nn.Dropout(p=0.5)

        self.fc2 = nn.Linear(in_features=512, out_features=10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)

        x = self.silu(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.bn2(x)

        x = self.silu(x)

        x = self.pool2(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.dropout1(x)

        x = self.silu(x)
        x = self.pool3(x)

        x = x.view(x.size(0), -1)

        x = self.fc1(x)
        x = self.dropout1(x)
        x = self.silu(x)

        x = self.fc2(x)

        return x



####Training and Testing on Model 1 with Silu activation and ADAM optimizer

In [ ]:

from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms


# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 25
accs = []

for seed in range(25):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = NetSilu()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70, 90], gamma=0.1)

    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()

    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))


Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.052466
Train Epoch: 5 [0/50 (0%)]	Loss: 0.398365
Train Epoch: 10 [0/50 (0%)]	Loss: 0.715135
Train Epoch: 15 [0/50 (0%)]	Loss: 0.382740
Train Epoch: 20 [0/50 (0%)]	Loss: 0.327082
Train Epoch: 25 [0/50 (0%)]	Loss: 0.183529
Train Epoch: 30 [0/50 (0%)]	Loss: 0.121717
Train Epoch: 35 [0/50 (0%)]	Loss: 0.087313
Train Epoch: 40 [0/50 (0%)]	Loss: 0.063626
Train Epoch: 45 [0/50 (0%)]	Loss: 0.032317
Train Epoch: 50 [0/50 (0%)]	Loss: 0.010000
Train Epoch: 55 [0/50 (0%)]	Loss: 0.046581
Train Epoch: 60 [0/50 (0%)]	Loss: 0.048812
Train Epoch: 65 [0/50 (0%)]	Loss: 0.038604
Train Epoch: 70 [0/50 (0%)]	Loss: 0.022041
Train Epoch: 75 [0/50 (0%)]	Loss: 0.054416
Train Epoch: 80 [0/50 (0%)]	Loss: 0.034324
Train Epoch: 85 [0/50 (0%)]	Loss: 0.025404
Train Epoch: 90 [0/50 (0%)]	Loss: 0.049592
Train Epoch: 95 [0/50 (0%)]	Loss: 0.034137

Test set: Ave

#####MODEL 1 performs better with the combination of silu activation and adam optimizer than silu activatino and Adadelta optimizer

####Training and testing on Model 1 with Silu activation and ADADELTA optimizer

In [ ]:

from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms


# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 25
accs = []

for seed in range(25):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = NetSilu()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adadelta(model.parameters(), lr=0.001, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70, 90], gamma=0.1)

    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()

    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))


Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.350815
Train Epoch: 5 [0/50 (0%)]	Loss: 2.376542
Train Epoch: 10 [0/50 (0%)]	Loss: 2.171153
Train Epoch: 15 [0/50 (0%)]	Loss: 2.096303
Train Epoch: 20 [0/50 (0%)]	Loss: 2.140448
Train Epoch: 25 [0/50 (0%)]	Loss: 1.957808
Train Epoch: 30 [0/50 (0%)]	Loss: 1.849039
Train Epoch: 35 [0/50 (0%)]	Loss: 1.797859
Train Epoch: 40 [0/50 (0%)]	Loss: 1.775832
Train Epoch: 45 [0/50 (0%)]	Loss: 1.819882
Train Epoch: 50 [0/50 (0%)]	Loss: 1.753970
Train Epoch: 55 [0/50 (0%)]	Loss: 1.676513
Train Epoch: 60 [0/50 (0%)]	Loss: 1.661655
Train Epoch: 65 [0/50 (0%)]	Loss: 1.704484
Train Epoch: 70 [0/50 (0%)]	Loss: 1.641617
Train Epoch: 75 [0/50 (0%)]	Loss: 1.741813
Train Epoch: 80 [0/50 (0%)]	Loss: 1.749309
Train Epoch: 85 [0/50 (0%)]	Loss: 1.688439
Train Epoch: 90 [0/50 (0%)]	Loss: 1.666979
Train Epoch: 95 [0/50 (0%)]	Loss: 1.839722


/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 1.9204, Accuracy: 310/400 (77.50%)

Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.532814
Train Epoch: 5 [0/50 (0%)]	Loss: 2.410903
Train Epoch: 10 [0/50 (0%)]	Loss: 2.378683
Train Epoch: 15 [0/50 (0%)]	Loss: 2.290132
Train Epoch: 20 [0/50 (0%)]	Loss: 2.170121
Train Epoch: 25 [0/50 (0%)]	Loss: 1.992010
Train Epoch: 30 [0/50 (0%)]	Loss: 2.006402
Train Epoch: 35 [0/50 (0%)]	Loss: 1.865653
Train Epoch: 40 [0/50 (0%)]	Loss: 1.800642
Train Epoch: 45 [0/50 (0%)]	Loss: 1.872011
Train Epoch: 50 [0/50 (0%)]	Loss: 1.879446
Train Epoch: 55 [0/50 (0%)]	Loss: 1.929063
Train Epoch: 60 [0/50 (0%)]	Loss: 1.742396
Train Epoch: 65 [0/50 (0%)]	Loss: 1.867386
Train Epoch: 70 [0/50 (0%)]	Loss: 1.750611
Train Epoch: 75 [0/50 (0%)]	Loss: 1.798714
Train Epoch: 80 [0/50 (0%)]	Loss: 1.733464
Train Epoch: 85 [0/50 (0%)]	Loss: 1.803825
Train Epoch: 90 [0/50 (0%)]	Loss: 1.747025
Train Epoch: 95 [0/50 (0%)]	Loss: 1.822660

Test set: Average loss: 1.98

#MODEL 2

###Model 2 with Relu activation(2 Convolution layers and 2 fully connected layers)

In [ ]:
import torch
import torch.nn as nn

class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()

        # Define the layers of the network
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(in_features=64 * 8 * 8, out_features=512)
        self.dropout1 = nn.Dropout(p=0.5)
        self.relu3 = nn.ReLU(inplace=True)

        self.fc2 = nn.Linear(in_features=512, out_features=10)

    def forward(self, x):
        # Pass input through the layers of the network
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = x.view(x.size(0), -1)

        x = self.fc1(x)
        x = self.dropout1(x)
        x = self.relu3(x)

        x = self.fc2(x)

        return x


####Training and Testing on Model 2 with ADADELTA optimizer and relu activation

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms

# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 25
accs = []

for seed in range(num_models):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = Net1()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adadelta(model.parameters(), lr=0.1, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70, 90], gamma=0.1)

    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()

    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))

Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.383142
Train Epoch: 5 [0/50 (0%)]	Loss: 1.084968
Train Epoch: 10 [0/50 (0%)]	Loss: 0.625074
Train Epoch: 15 [0/50 (0%)]	Loss: 0.610472
Train Epoch: 20 [0/50 (0%)]	Loss: 0.463523
Train Epoch: 25 [0/50 (0%)]	Loss: 0.348379
Train Epoch: 30 [0/50 (0%)]	Loss: 0.451932
Train Epoch: 35 [0/50 (0%)]	Loss: 0.318138
Train Epoch: 40 [0/50 (0%)]	Loss: 0.191503
Train Epoch: 45 [0/50 (0%)]	Loss: 0.206417
Train Epoch: 50 [0/50 (0%)]	Loss: 0.184189
Train Epoch: 55 [0/50 (0%)]	Loss: 0.237290
Train Epoch: 60 [0/50 (0%)]	Loss: 0.227466
Train Epoch: 65 [0/50 (0%)]	Loss: 0.166290
Train Epoch: 70 [0/50 (0%)]	Loss: 0.232505
Train Epoch: 75 [0/50 (0%)]	Loss: 0.192130
Train Epoch: 80 [0/50 (0%)]	Loss: 0.193304
Train Epoch: 85 [0/50 (0%)]	Loss: 0.241746
Train Epoch: 90 [0/50 (0%)]	Loss: 0.181680
Train Epoch: 95 [0/50 (0%)]	Loss: 0.168497

Test set: Ave

####Traing and Testing on Model 2 with ADAM optimizer and relu activation function.

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms

# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 25
accs = []

for seed in range(num_models):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = Net1()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70, 90], gamma=0.1)

    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()

    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))

Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.657106
Train Epoch: 5 [0/50 (0%)]	Loss: 59.623913
Train Epoch: 10 [0/50 (0%)]	Loss: 2.355735
Train Epoch: 15 [0/50 (0%)]	Loss: 0.687178
Train Epoch: 20 [0/50 (0%)]	Loss: 0.683921
Train Epoch: 25 [0/50 (0%)]	Loss: 0.667253
Train Epoch: 30 [0/50 (0%)]	Loss: 0.623559
Train Epoch: 35 [0/50 (0%)]	Loss: 0.646286
Train Epoch: 40 [0/50 (0%)]	Loss: 0.639857
Train Epoch: 45 [0/50 (0%)]	Loss: 0.577441
Train Epoch: 50 [0/50 (0%)]	Loss: 0.577465
Train Epoch: 55 [0/50 (0%)]	Loss: 0.542769
Train Epoch: 60 [0/50 (0%)]	Loss: 0.483442
Train Epoch: 65 [0/50 (0%)]	Loss: 0.493860
Train Epoch: 70 [0/50 (0%)]	Loss: 0.480995
Train Epoch: 75 [0/50 (0%)]	Loss: 0.556247
Train Epoch: 80 [0/50 (0%)]	Loss: 0.460763
Train Epoch: 85 [0/50 (0%)]	Loss: 0.504008
Train Epoch: 90 [0/50 (0%)]	Loss: 0.564576
Train Epoch: 95 [0/50 (0%)]	Loss: 0.463116

Test set: Av

###Model 2 with Mish activation function(2 convolution layers and 2 fully connected layers

In [ ]:
import torch
import torch.nn as nn

class Net1Mish(nn.Module):
    def __init__(self):
        super(Net1Mish, self).__init__()

        # Define the layers of the network
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.mish=nn.Mish()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(in_features=64 * 8 * 8, out_features=512)
        self.dropout1 = nn.Dropout(p=0.5)

        self.fc2 = nn.Linear(in_features=512, out_features=10)

    def forward(self, x):
        # Pass input through the layers of the network
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.mish(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.mish(x)
        x = self.pool2(x)

        x = x.view(x.size(0), -1)

        x = self.fc1(x)
        x = self.dropout1(x)
        x = self.mish(x)

        x = self.fc2(x)

        return x


####Training and Testing on Model 2 with Mish Activation function and ADADELTA optimizer

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms

# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 25
accs = []

for seed in range(num_models):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = Net1Mish()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adadelta(model.parameters(), lr=0.1, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70, 90], gamma=0.1)

    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()

    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))

Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.410460
Train Epoch: 5 [0/50 (0%)]	Loss: 0.679609
Train Epoch: 10 [0/50 (0%)]	Loss: 0.538156
Train Epoch: 15 [0/50 (0%)]	Loss: 0.476054
Train Epoch: 20 [0/50 (0%)]	Loss: 0.330053
Train Epoch: 25 [0/50 (0%)]	Loss: 0.263793
Train Epoch: 30 [0/50 (0%)]	Loss: 0.187305
Train Epoch: 35 [0/50 (0%)]	Loss: 0.215408
Train Epoch: 40 [0/50 (0%)]	Loss: 0.201953
Train Epoch: 45 [0/50 (0%)]	Loss: 0.128579
Train Epoch: 50 [0/50 (0%)]	Loss: 0.124211
Train Epoch: 55 [0/50 (0%)]	Loss: 0.192561
Train Epoch: 60 [0/50 (0%)]	Loss: 0.091403
Train Epoch: 65 [0/50 (0%)]	Loss: 0.103088
Train Epoch: 70 [0/50 (0%)]	Loss: 0.161526
Train Epoch: 75 [0/50 (0%)]	Loss: 0.114917
Train Epoch: 80 [0/50 (0%)]	Loss: 0.077092
Train Epoch: 85 [0/50 (0%)]	Loss: 0.126280
Train Epoch: 90 [0/50 (0%)]	Loss: 0.112500
Train Epoch: 95 [0/50 (0%)]	Loss: 0.113166

Test set: Ave

####Training and Testing on Model 2 with Mish activation and Adam optimizer

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms

# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 5
accs = []

for seed in range(num_models):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = Net1Mish()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70, 90], gamma=0.1)

    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()

    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))

Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.296682
Train Epoch: 5 [0/50 (0%)]	Loss: 291.672180
Train Epoch: 10 [0/50 (0%)]	Loss: 232.162415
Train Epoch: 15 [0/50 (0%)]	Loss: 175.755508
Train Epoch: 20 [0/50 (0%)]	Loss: 84.530647
Train Epoch: 25 [0/50 (0%)]	Loss: 25.310884
Train Epoch: 30 [0/50 (0%)]	Loss: 21.910540
Train Epoch: 35 [0/50 (0%)]	Loss: 28.197821
Train Epoch: 40 [0/50 (0%)]	Loss: 6.092014
Train Epoch: 45 [0/50 (0%)]	Loss: 1.835016
Train Epoch: 50 [0/50 (0%)]	Loss: 5.401855
Train Epoch: 55 [0/50 (0%)]	Loss: 7.887922
Train Epoch: 60 [0/50 (0%)]	Loss: 10.349282
Train Epoch: 65 [0/50 (0%)]	Loss: 9.461575
Train Epoch: 70 [0/50 (0%)]	Loss: 0.889352
Train Epoch: 75 [0/50 (0%)]	Loss: 6.079845
Train Epoch: 80 [0/50 (0%)]	Loss: 1.808564
Train Epoch: 85 [0/50 (0%)]	Loss: 3.616157
Train Epoch: 90 [0/50 (0%)]	Loss: 1.060747
Train Epoch: 95 [0/50 (0%)]	Loss: 4.318623

Te

#Model 3

###Model 3 with Relu activation( 4 Convolutional layers and 2 fully connected layers)

In [ ]:
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()

        # Define the layers of the network
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu1 = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.relu2 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.relu3 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.relu4 = nn.ReLU(inplace=True)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(in_features=2048, out_features=512)
        self.dropout1 = nn.Dropout(p=0.5)
        self.relu5 = nn.ReLU(inplace=True)

        self.fc2 = nn.Linear(in_features=512, out_features=10)

    def forward(self, x):
        # Pass input through the layers of the network
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.pool1(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.pool2(x)

        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu4(x)
        x = self.pool3(x)

        x = x.view(x.size(0), -1)

        x = self.fc1(x)
        x = self.dropout1(x)
        x = self.relu5(x)

        x = self.fc2(x)

        return x


####Training and Testing on Model 3 with ADADELTA optimizer but using MultiStepLR learning scheduler

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms

# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models =25
accs = []

for seed in range(num_models):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = Net2()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adadelta(model.parameters(), lr=0.1, weight_decay=0.0005)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70, 90], gamma=0.1)


    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()


    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))

Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.517791
Train Epoch: 5 [0/50 (0%)]	Loss: 1.020930
Train Epoch: 10 [0/50 (0%)]	Loss: 0.607880
Train Epoch: 15 [0/50 (0%)]	Loss: 0.536471
Train Epoch: 20 [0/50 (0%)]	Loss: 0.389501
Train Epoch: 25 [0/50 (0%)]	Loss: 0.350511
Train Epoch: 30 [0/50 (0%)]	Loss: 0.266325
Train Epoch: 35 [0/50 (0%)]	Loss: 0.258508
Train Epoch: 40 [0/50 (0%)]	Loss: 0.243711
Train Epoch: 45 [0/50 (0%)]	Loss: 0.187455
Train Epoch: 50 [0/50 (0%)]	Loss: 0.139628
Train Epoch: 55 [0/50 (0%)]	Loss: 0.180740
Train Epoch: 60 [0/50 (0%)]	Loss: 0.154735
Train Epoch: 65 [0/50 (0%)]	Loss: 0.128064
Train Epoch: 70 [0/50 (0%)]	Loss: 0.163396
Train Epoch: 75 [0/50 (0%)]	Loss: 0.129188
Train Epoch: 80 [0/50 (0%)]	Loss: 0.161277
Train Epoch: 85 [0/50 (0%)]	Loss: 0.121149
Train Epoch: 90 [0/50 (0%)]	Loss: 0.188952
Train Epoch: 95 [0/50 (0%)]	Loss: 0.121419

Test set: Ave

####Training and Testing on Model 3 with ADADELTA optimizer but using StepLR learning scheduler

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms

# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 25
accs = []

for seed in range(num_models):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = Net2()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adadelta(model.parameters(), lr=0.1, weight_decay=0.0005)
    scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=7, gamma=0.1)

    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()

    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))

Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.419916
Train Epoch: 5 [0/50 (0%)]	Loss: 1.299093
Train Epoch: 10 [0/50 (0%)]	Loss: 0.555960
Train Epoch: 15 [0/50 (0%)]	Loss: 0.486771
Train Epoch: 20 [0/50 (0%)]	Loss: 0.473034
Train Epoch: 25 [0/50 (0%)]	Loss: 0.488149
Train Epoch: 30 [0/50 (0%)]	Loss: 0.464455
Train Epoch: 35 [0/50 (0%)]	Loss: 0.431568
Train Epoch: 40 [0/50 (0%)]	Loss: 0.472380
Train Epoch: 45 [0/50 (0%)]	Loss: 0.454836
Train Epoch: 50 [0/50 (0%)]	Loss: 0.541322
Train Epoch: 55 [0/50 (0%)]	Loss: 0.416164
Train Epoch: 60 [0/50 (0%)]	Loss: 0.510999
Train Epoch: 65 [0/50 (0%)]	Loss: 0.479085
Train Epoch: 70 [0/50 (0%)]	Loss: 0.505439
Train Epoch: 75 [0/50 (0%)]	Loss: 0.412080
Train Epoch: 80 [0/50 (0%)]	Loss: 0.463511
Train Epoch: 85 [0/50 (0%)]	Loss: 0.525984
Train Epoch: 90 [0/50 (0%)]	Loss: 0.501168
Train Epoch: 95 [0/50 (0%)]	Loss: 0.497480

Test set: Ave

####Training and Testing on Model 3 with ADADELTA optimizer but using CosineAnnealingLR learning scheduler

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms

# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 25
accs = []

for seed in range(num_models):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = Net2()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adadelta(model.parameters(), lr=0.1, weight_decay=0.0005)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)


    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()

    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))

Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.332146
Train Epoch: 5 [0/50 (0%)]	Loss: 0.605244
Train Epoch: 10 [0/50 (0%)]	Loss: 0.630664
Train Epoch: 15 [0/50 (0%)]	Loss: 0.554814
Train Epoch: 20 [0/50 (0%)]	Loss: 1.309256
Train Epoch: 25 [0/50 (0%)]	Loss: 0.382606
Train Epoch: 30 [0/50 (0%)]	Loss: 0.380416
Train Epoch: 35 [0/50 (0%)]	Loss: 0.272658
Train Epoch: 40 [0/50 (0%)]	Loss: 0.296786
Train Epoch: 45 [0/50 (0%)]	Loss: 0.190066
Train Epoch: 50 [0/50 (0%)]	Loss: 0.222034
Train Epoch: 55 [0/50 (0%)]	Loss: 0.200873
Train Epoch: 60 [0/50 (0%)]	Loss: 0.177477
Train Epoch: 65 [0/50 (0%)]	Loss: 0.131533
Train Epoch: 70 [0/50 (0%)]	Loss: 0.131075
Train Epoch: 75 [0/50 (0%)]	Loss: 0.111344
Train Epoch: 80 [0/50 (0%)]	Loss: 0.154809
Train Epoch: 85 [0/50 (0%)]	Loss: 0.100035
Train Epoch: 90 [0/50 (0%)]	Loss: 0.050184
Train Epoch: 95 [0/50 (0%)]	Loss: 0.044061

Test set: Ave

####Training and Testing on Model 3 with relu activation and ADAM optimizer

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms

# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 25
accs = []

for seed in range(num_models):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = Net2()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70, 90], gamma=0.1)


    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()


    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))

Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.292601
Train Epoch: 5 [0/50 (0%)]	Loss: 48.845043
Train Epoch: 10 [0/50 (0%)]	Loss: 6.274275
Train Epoch: 15 [0/50 (0%)]	Loss: 1.471946
Train Epoch: 20 [0/50 (0%)]	Loss: 0.607094
Train Epoch: 25 [0/50 (0%)]	Loss: 0.648045
Train Epoch: 30 [0/50 (0%)]	Loss: 0.715174
Train Epoch: 35 [0/50 (0%)]	Loss: 0.574210
Train Epoch: 40 [0/50 (0%)]	Loss: 0.703617
Train Epoch: 45 [0/50 (0%)]	Loss: 0.600606
Train Epoch: 50 [0/50 (0%)]	Loss: 0.673406
Train Epoch: 55 [0/50 (0%)]	Loss: 0.652310
Train Epoch: 60 [0/50 (0%)]	Loss: 0.651517
Train Epoch: 65 [0/50 (0%)]	Loss: 0.596068
Train Epoch: 70 [0/50 (0%)]	Loss: 0.537463
Train Epoch: 75 [0/50 (0%)]	Loss: 0.524242
Train Epoch: 80 [0/50 (0%)]	Loss: 0.513713
Train Epoch: 85 [0/50 (0%)]	Loss: 0.530640
Train Epoch: 90 [0/50 (0%)]	Loss: 0.575673
Train Epoch: 95 [0/50 (0%)]	Loss: 0.536827

Test set: Av

###Model 3 with Mish activation(4 Convolutional layers and 2 fully connected layers)

In [ ]:
class Net2Mish(nn.Module):
    def __init__(self):
        super(Net2Mish, self).__init__()

        # Define the layers of the network
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.mish=nn.Mish()

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(in_features=2048, out_features=512)
        self.dropout1 = nn.Dropout(p=0.5)

        self.fc2 = nn.Linear(in_features=512, out_features=10)

    def forward(self, x):
        # Pass input through the layers of the network
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.mish(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.mish(x)
        x = self.pool1(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.mish(x)
        x = self.pool2(x)

        x = self.conv4(x)
        x = self.bn4(x)
        x = self.mish(x)
        x = self.pool3(x)

        x = x.view(x.size(0), -1)

        x = self.fc1(x)
        x = self.dropout1(x)
        x = self.mish(x)

        x = self.fc2(x)

        return x


####Training and Testing on Model 3 with Mish activation funcition and ADADELTA optimizer


In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms

# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 25
accs = []

for seed in range(num_models):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = Net2Mish()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adadelta(model.parameters(), lr=0.1, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70, 90], gamma=0.1)


    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()


    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))

Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.466724
Train Epoch: 5 [0/50 (0%)]	Loss: 0.750491
Train Epoch: 10 [0/50 (0%)]	Loss: 0.476603
Train Epoch: 15 [0/50 (0%)]	Loss: 0.390994
Train Epoch: 20 [0/50 (0%)]	Loss: 0.283154
Train Epoch: 25 [0/50 (0%)]	Loss: 0.183159
Train Epoch: 30 [0/50 (0%)]	Loss: 0.095988
Train Epoch: 35 [0/50 (0%)]	Loss: 0.113922
Train Epoch: 40 [0/50 (0%)]	Loss: 0.089293
Train Epoch: 45 [0/50 (0%)]	Loss: 0.054852
Train Epoch: 50 [0/50 (0%)]	Loss: 0.069675
Train Epoch: 55 [0/50 (0%)]	Loss: 0.056770
Train Epoch: 60 [0/50 (0%)]	Loss: 0.061185
Train Epoch: 65 [0/50 (0%)]	Loss: 0.042757
Train Epoch: 70 [0/50 (0%)]	Loss: 0.075722
Train Epoch: 75 [0/50 (0%)]	Loss: 0.065157
Train Epoch: 80 [0/50 (0%)]	Loss: 0.060626
Train Epoch: 85 [0/50 (0%)]	Loss: 0.053498
Train Epoch: 90 [0/50 (0%)]	Loss: 0.046062
Train Epoch: 95 [0/50 (0%)]	Loss: 0.079005

Test set: Ave

####Training and Testing on Model 3 with Mish activation funcition and ADAM optimizer


In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms

# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 25
accs = []

for seed in range(num_models):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = Net2Mish()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70, 90], gamma=0.1)


    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()


    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))

Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.295637
Train Epoch: 5 [0/50 (0%)]	Loss: 295.101654
Train Epoch: 10 [0/50 (0%)]	Loss: 94.400871
Train Epoch: 15 [0/50 (0%)]	Loss: 42.304169
Train Epoch: 20 [0/50 (0%)]	Loss: 15.810637
Train Epoch: 25 [0/50 (0%)]	Loss: 13.021039
Train Epoch: 30 [0/50 (0%)]	Loss: 3.561769
Train Epoch: 35 [0/50 (0%)]	Loss: 1.906255
Train Epoch: 40 [0/50 (0%)]	Loss: 0.994172
Train Epoch: 45 [0/50 (0%)]	Loss: 0.975116
Train Epoch: 50 [0/50 (0%)]	Loss: 1.153436
Train Epoch: 55 [0/50 (0%)]	Loss: 0.697984
Train Epoch: 60 [0/50 (0%)]	Loss: 0.823258
Train Epoch: 65 [0/50 (0%)]	Loss: 0.595639
Train Epoch: 70 [0/50 (0%)]	Loss: 0.486298
Train Epoch: 75 [0/50 (0%)]	Loss: 0.569786
Train Epoch: 80 [0/50 (0%)]	Loss: 0.399596
Train Epoch: 85 [0/50 (0%)]	Loss: 0.480179
Train Epoch: 90 [0/50 (0%)]	Loss: 0.519058
Train Epoch: 95 [0/50 (0%)]	Loss: 0.400816

Test se

###Model 3 with Silu activation(4 Convolutional layers and 2 fully connected layers)

In [ ]:
class Net2silu(nn.Module):
    def __init__(self):
        super(Net2silu, self).__init__()

        # Define the layers of the network
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.Silu=nn.SiLU()

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(in_features=2048, out_features=512)
        self.dropout1 = nn.Dropout(p=0.5)

        self.fc2 = nn.Linear(in_features=512, out_features=10)

    def forward(self, x):
        # Pass input through the layers of the network
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.Silu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.Silu(x)
        x = self.pool1(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.Silu(x)
        x = self.pool2(x)

        x = self.conv4(x)
        x = self.bn4(x)
        x = self.Silu(x)
        x = self.pool3(x)

        x = x.view(x.size(0), -1)

        x = self.fc1(x)
        x = self.dropout1(x)
        x = self.Silu(x)

        x = self.fc2(x)

        return x


####Training and Testing on Model 3 with SiLU activation and ADADELTA Optimizer

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms

# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 25
accs = []

for seed in range(num_models):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = Net2silu()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adadelta(model.parameters(), lr=0.1, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70, 90], gamma=0.1)


    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()


    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))

Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.363544
Train Epoch: 5 [0/50 (0%)]	Loss: 0.699976
Train Epoch: 10 [0/50 (0%)]	Loss: 0.598551
Train Epoch: 15 [0/50 (0%)]	Loss: 0.478085
Train Epoch: 20 [0/50 (0%)]	Loss: 0.273494
Train Epoch: 25 [0/50 (0%)]	Loss: 0.214987
Train Epoch: 30 [0/50 (0%)]	Loss: 0.126268
Train Epoch: 35 [0/50 (0%)]	Loss: 0.180305
Train Epoch: 40 [0/50 (0%)]	Loss: 0.103719
Train Epoch: 45 [0/50 (0%)]	Loss: 0.066825
Train Epoch: 50 [0/50 (0%)]	Loss: 0.056519
Train Epoch: 55 [0/50 (0%)]	Loss: 0.072116
Train Epoch: 60 [0/50 (0%)]	Loss: 0.041542
Train Epoch: 65 [0/50 (0%)]	Loss: 0.048468
Train Epoch: 70 [0/50 (0%)]	Loss: 0.038722
Train Epoch: 75 [0/50 (0%)]	Loss: 0.059441
Train Epoch: 80 [0/50 (0%)]	Loss: 0.088612
Train Epoch: 85 [0/50 (0%)]	Loss: 0.057988
Train Epoch: 90 [0/50 (0%)]	Loss: 0.045469
Train Epoch: 95 [0/50 (0%)]	Loss: 0.055968

Test set: Ave

####Training and Testing on Model 3 with SiLU activation and ADAM optimizer

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms

# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 25
accs = []

for seed in range(num_models):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = Net2silu()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70, 90], gamma=0.1)


    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()


    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))

Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.285597
Train Epoch: 5 [0/50 (0%)]	Loss: 61.193226
Train Epoch: 10 [0/50 (0%)]	Loss: 46.465088
Train Epoch: 15 [0/50 (0%)]	Loss: 23.663034
Train Epoch: 20 [0/50 (0%)]	Loss: 31.147041
Train Epoch: 25 [0/50 (0%)]	Loss: 17.940285
Train Epoch: 30 [0/50 (0%)]	Loss: 17.941132
Train Epoch: 35 [0/50 (0%)]	Loss: 3.973277
Train Epoch: 40 [0/50 (0%)]	Loss: 9.017503
Train Epoch: 45 [0/50 (0%)]	Loss: 6.370782
Train Epoch: 50 [0/50 (0%)]	Loss: 5.231541
Train Epoch: 55 [0/50 (0%)]	Loss: 2.646484
Train Epoch: 60 [0/50 (0%)]	Loss: 3.157420
Train Epoch: 65 [0/50 (0%)]	Loss: 6.409414
Train Epoch: 70 [0/50 (0%)]	Loss: 1.382933
Train Epoch: 75 [0/50 (0%)]	Loss: 4.947385
Train Epoch: 80 [0/50 (0%)]	Loss: 3.007179
Train Epoch: 85 [0/50 (0%)]	Loss: 2.686731
Train Epoch: 90 [0/50 (0%)]	Loss: 1.465506
Train Epoch: 95 [0/50 (0%)]	Loss: 0.354097

Test se

#MODEL 4

###Model 4 with Relu activation (6 Convolutional layers and 4 fully connected layers)

In [ ]:
import torch.nn as nn

class Net4(nn.Module):
    def __init__(self):
        super(Net4, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(512, 1024, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(1024, 1024, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(1024 * 4 * 4, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 2048)
        self.fc4 = nn.Linear(2048, 10)

        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(p=0.5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.relu(self.conv4(x))
        x = self.pool(x)
        x = self.relu(self.conv5(x))
        x = self.relu(self.conv6(x))
        x = self.pool(x)
        x = x.view(-1, 1024 * 4 * 4)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.dropout(self.relu(self.fc3(x)))
        x = self.fc4(x)
        return x


####Training and Testing on Model 4 with Relu activation and ADADELTA optimizer

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms

# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 25
accs = []

for seed in range(num_models):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = Net4()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adadelta(model.parameters(), lr=0.1, weight_decay=0.0005)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70, 90], gamma=0.1)

    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()

    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))

Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.299192
Train Epoch: 5 [0/50 (0%)]	Loss: 2.261801
Train Epoch: 10 [0/50 (0%)]	Loss: 2.221802
Train Epoch: 15 [0/50 (0%)]	Loss: 2.154298
Train Epoch: 20 [0/50 (0%)]	Loss: 1.907558
Train Epoch: 25 [0/50 (0%)]	Loss: 0.934918
Train Epoch: 30 [0/50 (0%)]	Loss: 0.887352
Train Epoch: 35 [0/50 (0%)]	Loss: 0.757485
Train Epoch: 40 [0/50 (0%)]	Loss: 0.815285
Train Epoch: 45 [0/50 (0%)]	Loss: 0.711584
Train Epoch: 50 [0/50 (0%)]	Loss: 0.739408
Train Epoch: 55 [0/50 (0%)]	Loss: 0.717575
Train Epoch: 60 [0/50 (0%)]	Loss: 0.727269
Train Epoch: 65 [0/50 (0%)]	Loss: 0.706852
Train Epoch: 70 [0/50 (0%)]	Loss: 0.713909
Train Epoch: 75 [0/50 (0%)]	Loss: 0.714096
Train Epoch: 80 [0/50 (0%)]	Loss: 0.696052
Train Epoch: 85 [0/50 (0%)]	Loss: 0.721042
Train Epoch: 90 [0/50 (0%)]	Loss: 0.714718
Train Epoch: 95 [0/50 (0%)]	Loss: 0.736548

Test set: Ave

####Training and Testing on Model 4 with Relu activation function and ADAM optimizer

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms

# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 25
accs = []

for seed in range(num_models):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = Net4()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70, 90], gamma=0.1)

    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()

    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))

Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.298014
Train Epoch: 5 [0/50 (0%)]	Loss: 22963263488.000000
Train Epoch: 10 [0/50 (0%)]	Loss: 1202967552.000000
Train Epoch: 15 [0/50 (0%)]	Loss: 842852096.000000
Train Epoch: 20 [0/50 (0%)]	Loss: 117374312.000000
Train Epoch: 25 [0/50 (0%)]	Loss: 37713748.000000
Train Epoch: 30 [0/50 (0%)]	Loss: 190242816.000000
Train Epoch: 35 [0/50 (0%)]	Loss: 136604288.000000
Train Epoch: 40 [0/50 (0%)]	Loss: 102416016.000000
Train Epoch: 45 [0/50 (0%)]	Loss: 73512424.000000
Train Epoch: 50 [0/50 (0%)]	Loss: 60161884.000000
Train Epoch: 55 [0/50 (0%)]	Loss: 20613250.000000
Train Epoch: 60 [0/50 (0%)]	Loss: 45775704.000000
Train Epoch: 65 [0/50 (0%)]	Loss: 24360072.000000
Train Epoch: 70 [0/50 (0%)]	Loss: 28503834.000000
Train Epoch: 75 [0/50 (0%)]	Loss: 27049272.000000
Train Epoch: 80 [0/50 (0%)]	Loss: 25194278.000000
Train Epoch: 85 [0/50

###Model 4 with Mish Activation Function(6 convolutional layers and 4 fully connected layers)

In [ ]:
import torch.nn as nn

class Net4Mish(nn.Module):
    def __init__(self):
        super(Net4Mish, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(512, 1024, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(1024, 1024, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(1024 * 4 * 4, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 2048)
        self.fc4 = nn.Linear(2048, 10)

        self.mish = nn.Mish()
        self.dropout = nn.Dropout(p=0.5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.mish(self.conv1(x))
        x = self.mish(self.conv2(x))
        x = self.pool(x)
        x = self.mish(self.conv3(x))
        x = self.mish(self.conv4(x))
        x = self.pool(x)
        x = self.mish(self.conv5(x))
        x = self.mish(self.conv6(x))
        x = self.pool(x)
        x = x.view(-1, 1024 * 4 * 4)
        x = self.dropout(self.mish(self.fc1(x)))
        x = self.dropout(self.mish(self.fc2(x)))
        x = self.dropout(self.mish(self.fc3(x)))
        x = self.fc4(x)
        return x


####Training and Testing on Model 4 with Mish activation and ADADELTA optimizer

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms

# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 25
accs = []

for seed in range(num_models):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = Net4Mish()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adadelta(model.parameters(), lr=0.1, weight_decay=0.0005)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70, 90], gamma=0.1)

    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()

    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))

Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.293612
Train Epoch: 5 [0/50 (0%)]	Loss: 2.262512
Train Epoch: 10 [0/50 (0%)]	Loss: 2.232021
Train Epoch: 15 [0/50 (0%)]	Loss: 2.195803
Train Epoch: 20 [0/50 (0%)]	Loss: 2.153905
Train Epoch: 25 [0/50 (0%)]	Loss: 2.099244
Train Epoch: 30 [0/50 (0%)]	Loss: 2.032079
Train Epoch: 35 [0/50 (0%)]	Loss: 1.945674
Train Epoch: 40 [0/50 (0%)]	Loss: 1.836903
Train Epoch: 45 [0/50 (0%)]	Loss: 1.818999
Train Epoch: 50 [0/50 (0%)]	Loss: 1.808713
Train Epoch: 55 [0/50 (0%)]	Loss: 1.789115
Train Epoch: 60 [0/50 (0%)]	Loss: 1.781627
Train Epoch: 65 [0/50 (0%)]	Loss: 1.766003
Train Epoch: 70 [0/50 (0%)]	Loss: 1.749187
Train Epoch: 75 [0/50 (0%)]	Loss: 1.752701
Train Epoch: 80 [0/50 (0%)]	Loss: 1.747497
Train Epoch: 85 [0/50 (0%)]	Loss: 1.747996
Train Epoch: 90 [0/50 (0%)]	Loss: 1.747410
Train Epoch: 95 [0/50 (0%)]	Loss: 1.742921

Test set: Ave

####Training and Testing on Model 4 with Mish Activation and ADAM optimizer

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms

# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 25
accs = []

for seed in range(num_models):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = Net4Mish()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=0.0005)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70, 90], gamma=0.1)

    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()

    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))

Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.290344
Train Epoch: 5 [0/50 (0%)]	Loss: 15302561527496704.000000
Train Epoch: 10 [0/50 (0%)]	Loss: 102686752768.000000
Train Epoch: 15 [0/50 (0%)]	Loss: 1865338978304.000000
Train Epoch: 20 [0/50 (0%)]	Loss: 6.312678
Train Epoch: 25 [0/50 (0%)]	Loss: 3.602964
Train Epoch: 30 [0/50 (0%)]	Loss: 4.384400
Train Epoch: 35 [0/50 (0%)]	Loss: 3.103721
Train Epoch: 40 [0/50 (0%)]	Loss: 5.315300
Train Epoch: 45 [0/50 (0%)]	Loss: 2.417793
Train Epoch: 50 [0/50 (0%)]	Loss: 4.486658
Train Epoch: 55 [0/50 (0%)]	Loss: 4.691803
Train Epoch: 60 [0/50 (0%)]	Loss: 3.103549
Train Epoch: 65 [0/50 (0%)]	Loss: 3.125655
Train Epoch: 70 [0/50 (0%)]	Loss: 3.251859
Train Epoch: 75 [0/50 (0%)]	Loss: 4.472920
Train Epoch: 80 [0/50 (0%)]	Loss: 2.729499
Train Epoch: 85 [0/50 (0%)]	Loss: 2.993827
Train Epoch: 90 [0/50 (0%)]	Loss: 3.145162
Train Epoch: 95 [0

#MODEL 5

###Model 5 with Relu activation and batch normalization (6 Convolutional layers and 4 fully connected layers)

In [2]:
import torch.nn as nn

class Net5(nn.Module):
    def __init__(self):
        super(Net5, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.conv5 = nn.Conv2d(512, 1024, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(1024)
        self.conv6 = nn.Conv2d(1024, 1024, kernel_size=3, padding=1)
        self.bn6 = nn.BatchNorm2d(1024)
        self.fc1 = nn.Linear(1024 * 4 * 4, 4096)
        self.bn_fc1 = nn.BatchNorm1d(4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.bn_fc2 = nn.BatchNorm1d(4096)
        self.fc3 = nn.Linear(4096, 2048)
        self.bn_fc3 = nn.BatchNorm1d(2048)
        self.fc4 = nn.Linear(2048, 10)

        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(p=0.5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.pool(x)
        x = self.relu(self.bn5(self.conv5(x)))
        x = self.relu(self.bn6(self.conv6(x)))
        x = self.pool(x)
        x = x.view(-1, 1024 * 4 * 4)
        x = self.dropout(self.relu(self.bn_fc1(self.fc1(x))))
        x = self.dropout(self.relu(self.bn_fc2(self.fc2(x))))
        x = self.dropout(self.relu(self.bn_fc3(self.fc3(x))))
        x = self.fc4(x)
        return x


####Training and Testing on Model 5 with Relu activation function ADADELTA optimizer

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms

# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 25
accs = []

for seed in range(num_models):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = Net5()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adadelta(model.parameters(), lr=0.1, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70, 90], gamma=0.1)

    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()

    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))

Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.626025
Train Epoch: 5 [0/50 (0%)]	Loss: 0.477268
Train Epoch: 10 [0/50 (0%)]	Loss: 0.192487
Train Epoch: 15 [0/50 (0%)]	Loss: 0.084457
Train Epoch: 20 [0/50 (0%)]	Loss: 0.029571
Train Epoch: 25 [0/50 (0%)]	Loss: 0.019456
Train Epoch: 30 [0/50 (0%)]	Loss: 0.011488
Train Epoch: 35 [0/50 (0%)]	Loss: 0.009344
Train Epoch: 40 [0/50 (0%)]	Loss: 0.010824
Train Epoch: 45 [0/50 (0%)]	Loss: 0.008583
Train Epoch: 50 [0/50 (0%)]	Loss: 0.006400
Train Epoch: 55 [0/50 (0%)]	Loss: 0.007810
Train Epoch: 60 [0/50 (0%)]	Loss: 0.007838
Train Epoch: 65 [0/50 (0%)]	Loss: 0.009162
Train Epoch: 70 [0/50 (0%)]	Loss: 0.008426
Train Epoch: 75 [0/50 (0%)]	Loss: 0.006287
Train Epoch: 80 [0/50 (0%)]	Loss: 0.006506
Train Epoch: 85 [0/50 (0%)]	Loss: 0.008878
Train Epoch: 90 [0/50 (0%)]	Loss: 0.010047
Train Epoch: 95 [0/50 (0%)]	Loss: 0.008291

Test set: Ave

####Training and Testing on Model 5 with relu activation and ADAM optimizer

In [4]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms
import torch

# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 25
accs = []

for seed in range(num_models):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = Net5()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70, 90], gamma=0.1)

    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()

    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))

100%|██████████| 170498071/170498071 [00:02<00:00, 76390697.25it/s] 


Extracting ./cifar-10-python.tar.gz to .
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.375575
Train Epoch: 5 [0/50 (0%)]	Loss: 12.472812
Train Epoch: 10 [0/50 (0%)]	Loss: 6.706880
Train Epoch: 15 [0/50 (0%)]	Loss: 3.755360
Train Epoch: 20 [0/50 (0%)]	Loss: 2.600731
Train Epoch: 25 [0/50 (0%)]	Loss: 3.770291
Train Epoch: 30 [0/50 (0%)]	Loss: 1.164671
Train Epoch: 35 [0/50 (0%)]	Loss: 0.824958
Train Epoch: 40 [0/50 (0%)]	Loss: 0.726681
Train Epoch: 45 [0/50 (0%)]	Loss: 0.494929
Train Epoch: 50 [0/50 (0%)]	Loss: 0.496585
Train Epoch: 55 [0/50 (0%)]	Loss: 0.825092
Train Epoch: 60 [0/50 (0%)]	Loss: 0.449781
Train Epoch: 65 [0/50 (0%)]	Loss: 0.446600
Train Epoch: 70 [0/50 (0%)]	Loss: 0.436079
Train Epoch: 75 [0/50 (0%)]	Loss: 0.387967
Train Epoch: 80 [0/50 (0%)]	Loss: 0.497475
Train Epoch: 85 [0/50 (0%)]	Loss: 0.466792
Train Epoch: 90 [0/50 (0%)]	Loss: 0.431116
Train Epoch: 95 [0/50 (0%)]	Loss: 0.362806


/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 1.2871, Accuracy: 311/400 (77.75%)

Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.576598
Train Epoch: 5 [0/50 (0%)]	Loss: 5.559458
Train Epoch: 10 [0/50 (0%)]	Loss: 4.230481
Train Epoch: 15 [0/50 (0%)]	Loss: 1.753459
Train Epoch: 20 [0/50 (0%)]	Loss: 4.843584
Train Epoch: 25 [0/50 (0%)]	Loss: 1.183168
Train Epoch: 30 [0/50 (0%)]	Loss: 1.207366
Train Epoch: 35 [0/50 (0%)]	Loss: 1.547605
Train Epoch: 40 [0/50 (0%)]	Loss: 0.990679
Train Epoch: 45 [0/50 (0%)]	Loss: 0.770631
Train Epoch: 50 [0/50 (0%)]	Loss: 0.746212
Train Epoch: 55 [0/50 (0%)]	Loss: 0.864880
Train Epoch: 60 [0/50 (0%)]	Loss: 0.647185
Train Epoch: 65 [0/50 (0%)]	Loss: 0.684754
Train Epoch: 70 [0/50 (0%)]	Loss: 0.697670
Train Epoch: 75 [0/50 (0%)]	Loss: 0.726071
Train Epoch: 80 [0/50 (0%)]	Loss: 0.737181
Train Epoch: 85 [0/50 (0%)]	Loss: 0.720255
Train Epoch: 90 [0/50 (0%)]	Loss: 0.697391
Train Epoch: 95 [0/50 (0%)]	Loss: 0.757948

Test set: Average loss: 0.80

###Model 5 with Mish activation (6 convolutional layers and 4 fully connected layers with batch Normalization)

In [ ]:
import torch.nn as nn

class Net5Mish(nn.Module):
    def __init__(self):
        super(Net5Mish, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.conv5 = nn.Conv2d(512, 1024, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(1024)
        self.conv6 = nn.Conv2d(1024, 1024, kernel_size=3, padding=1)
        self.bn6 = nn.BatchNorm2d(1024)
        self.fc1 = nn.Linear(1024 * 4 * 4, 4096)
        self.bn_fc1 = nn.BatchNorm1d(4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.bn_fc2 = nn.BatchNorm1d(4096)
        self.fc3 = nn.Linear(4096, 2048)
        self.bn_fc3 = nn.BatchNorm1d(2048)
        self.fc4 = nn.Linear(2048, 10)

        self.mish = nn.Mish()
        self.dropout = nn.Dropout(p=0.5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.mish(self.bn1(self.conv1(x)))
        x = self.mish(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.mish(self.bn3(self.conv3(x)))
        x = self.mish(self.bn4(self.conv4(x)))
        x = self.pool(x)
        x = self.mish(self.bn5(self.conv5(x)))
        x = self.mish(self.bn6(self.conv6(x)))
        x = self.pool(x)
        x = x.view(-1, 1024 * 4 * 4)
        x = self.dropout(self.mish(self.bn_fc1(self.fc1(x))))
        x = self.dropout(self.mish(self.bn_fc2(self.fc2(x))))
        x = self.dropout(self.mish(self.bn_fc3(self.fc3(x))))
        x = self.fc4(x)
        return x


####Training and Testing on Model 5 with Mish activation function and ADADELTA optimizer

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms

# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 25
accs = []

for seed in range(num_models):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = Net5Mish()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adadelta(model.parameters(), lr=0.1, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70, 90], gamma=0.1)

    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()

    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))

####Training and Testing on Model 5 with Mish activation and ADAM optimizer

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms

# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 25
accs = []

for seed in range(num_models):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = Net5Mish()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70, 90], gamma=0.1)

    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()

    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))

###Model 5 on SiLU activation function

In [ ]:
import torch.nn as nn

class Net5Silu(nn.Module):
    def __init__(self):
        super(Net5Silu, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.conv5 = nn.Conv2d(512, 1024, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(1024)
        self.conv6 = nn.Conv2d(1024, 1024, kernel_size=3, padding=1)
        self.bn6 = nn.BatchNorm2d(1024)
        self.fc1 = nn.Linear(1024 * 4 * 4, 4096)
        self.bn_fc1 = nn.BatchNorm1d(4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.bn_fc2 = nn.BatchNorm1d(4096)
        self.fc3 = nn.Linear(4096, 2048)
        self.bn_fc3 = nn.BatchNorm1d(2048)
        self.fc4 = nn.Linear(2048, 10)

        self.silu = nn.SiLU()
        self.dropout = nn.Dropout(p=0.5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.silu(self.bn1(self.conv1(x)))
        x = self.silu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.silu(self.bn3(self.conv3(x)))
        x = self.silu(self.bn4(self.conv4(x)))
        x = self.pool(x)
        x = self.silu(self.bn5(self.conv5(x)))
        x = self.silu(self.bn6(self.conv6(x)))
        x = self.pool(x)
        x = x.view(-1, 1024 * 4 * 4)
        x = self.dropout(self.silu(self.bn_fc1(self.fc1(x))))
        x = self.dropout(self.silu(self.bn_fc2(self.fc2(x))))
        x = self.dropout(self.silu(self.bn_fc3(self.fc3(x))))
        x = self.fc4(x)
        return x


####Training on Model 5 with SiLU activation function and ADADELTA optimizer

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms

# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 25
accs = []

for seed in range(num_models):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = Net5Silu()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adadelta(model.parameters(), lr=0.1, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70, 90], gamma=0.1)

    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()

    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))

####Training and testing on model 5 with SiLU activation function and ADAM optimizer

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
from torchvision import datasets, transforms

# Data augmentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# CIFAR-10 data
cifar_train = datasets.CIFAR10(root='.', train=True, transform=train_transform, download=True)
cifar_val = datasets.CIFAR10(root='.', train=True, transform=val_transform, download=True)

# Ensemble learning
num_models = 25
accs = []

for seed in range(num_models):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes = prng.permutation(np.arange(0, 10))
    indx_train = np.concatenate([np.where(np.array(cifar_train.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_train, indx_train)
    val_data = Subset(cifar_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    model = Net5Silu()
    model.to(device)

    # Learning rate scheduling
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70, 90], gamma=0.1)

    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch % 5 == 0)
        scheduler.step()

    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over %d instances: %.2f +- %.2f'%(num_models, accs.mean(), accs.std()))